In [105]:
import os
import sqlite3
import pandas as pd
from zipfile import ZipFile
from matplotlib import pyplot as plt

In [106]:
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), '..'))
ZIPPED_DATA_DIR = os.path.join(BASE_DIR, "zippedData")

In [141]:
dfs = {}

sql_db_file = "im.db"
movie_info_q = """
        SELECT * FROM principals
        JOIN movie_ratings USING(movie_id)
        JOIN movie_akas USING(movie_id)
        ;"""
movie_people_q = """
        SELECT *
        FROM principals
        JOIN persons USING(person_id)
        ;"""

with ZipFile(os.path.join(ZIPPED_DATA_DIR, f"{sql_db_file}.zip"), 'r') as zip:
    zip.extract(sql_db_file)

    conn = sqlite3.connect(sql_db_file)
    
    dfs["movie_info_df"] = pd.read_sql(movie_info_q, conn)
    dfs["movie_people_df"] = pd.read_sql(movie_people_q, conn)

    os.remove(sql_db_file)


for file_name in os.listdir(ZIPPED_DATA_DIR):
    if not file_name in [sql_db_file + ".zip", ".DS_Store"]:
        sep = "\t" if "tsv" in file_name else ","
        try:
            dfs[file_name.split(".")[-3]] = pd.read_csv(os.path.join(ZIPPED_DATA_DIR, file_name), sep=sep, encoding="unicode_escape")
        except Exception as e:
            print(e, file_name)

'unicodeescape' codec can't decode bytes in position 108-109: malformed \N character escape title.akas.tsv


In [171]:
dfs.keys()

dict_keys(['movie_info_df', 'movie_people_df', 'movies', 'reviews', 'movie_info', 'movie_budgets', 'movie_gross'])

In [174]:
for df in dfs:
    print(dfs.get(df).columns)

Index(['movie_id', 'ordering', 'person_id', 'category', 'job', 'characters',
       'averagerating', 'numvotes', 'ordering', 'title', 'region', 'language',
       'types', 'attributes', 'is_original_title'],
      dtype='object')
Index(['movie_id', 'ordering', 'person_id', 'category', 'job', 'characters',
       'primary_name', 'birth_year', 'death_year', 'primary_profession'],
      dtype='object')
Index(['Unnamed: 0', 'genre_ids', 'id', 'original_language', 'original_title',
       'popularity', 'release_date', 'title', 'vote_average', 'vote_count'],
      dtype='object')
Index(['id', 'review', 'rating', 'fresh', 'critic', 'top_critic', 'publisher',
       'date'],
      dtype='object')
Index(['id', 'synopsis', 'rating', 'genre', 'director', 'writer',
       'theater_date', 'dvd_date', 'currency', 'box_office', 'runtime',
       'studio'],
      dtype='object')
Index(['id', 'release_date', 'movie', 'production_budget', 'domestic_gross',
       'worldwide_gross'],
      dtype='object'